In [ ]:
from LookGenerator.datasets.utils import prepare_image_for_encoder, to_array_from_decoder

In [ ]:
#всякие загрузки картинок
human_image = # Image
pose_points = # array
clothes = # Image
transform_input = #

In [ ]:
to_model = prepare_image_for_encoder(human_image, pose_points, clothes, transform_input)

In [ ]:
#загрузочно моделочная энкодерно декодерная магия

In [ ]:
modelled = model(to_model)

In [ ]:
modelled = to_array_from_decoder(modelled)